In [1]:
import pandas as pd
import numpy as np 
import os
import shutil
import torch

In [2]:
Base_path_to_datas_to_train = r"C:\Users\Asus\Documents\Ship Classification\flask\input\train"
Base_path_to_datas_to_val = r"C:\Users\Asus\Documents\Ship Classification\flask\input\valid"
Base_path_to_datas_to_test = r"C:\Users\Asus\Documents\Ship Classification\flask\input\test"


PIN_MEMORY=False
#=======================================================================
clas_of_img={}
#=======================================================================
results_dict = {}
modeli=[]
#=======================================================================
torch.manual_seed(5017)
torch.cuda.manual_seed(5017)

In [3]:
def gen_classes(path,class_dict):
    for indx, path in enumerate(os.listdir(Base_path_to_datas_to_train)):
        class_dict[indx] = path
        
        
    return len(class_dict)
        

In [4]:
Num_classes = gen_classes(Base_path_to_datas_to_train,clas_of_img)
clas_of_img

{0: 'Aircraft Carrier',
 1: 'Bulkers',
 2: 'Car Carrier',
 3: 'Container Ship',
 4: 'Cruise',
 5: 'DDG',
 6: 'Recreational',
 7: 'Sailboat',
 8: 'Submarine',
 9: 'Tug'}

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator# type: ignore
from keras.applications.vgg16 import preprocess_input# type: ignore
from tensorflow.keras.applications import VGG16# type: ignore
     
train_datagen = ImageDataGenerator(
    rotation_range=45,
    horizontal_flip=True,
    width_shift_range=0.5,
    height_shift_range=0.5,
    validation_split=0.2,
    preprocessing_function=preprocess_input
)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)


In [6]:
train_set = train_datagen.flow_from_directory(Base_path_to_datas_to_train, batch_size=32, target_size=(224,224))


Found 8536 images belonging to 10 classes.


In [7]:
validation_set = train_datagen.flow_from_directory(Base_path_to_datas_to_val, batch_size=32,target_size=(224,224),shuffle=False)


Found 942 images belonging to 10 classes.


In [8]:
test_set = test_datagen.flow_from_directory(Base_path_to_datas_to_test, batch_size=32,target_size=(224,224))

Found 781 images belonging to 10 classes.


In [12]:
from tensorflow.keras.models import load_model
model=load_model(r'C:\Users\Asus\Documents\Ship Classification\flask\notebook\vgg16-ship-calssification.h5')

In [35]:
print(validation_set.classes)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 

In [36]:
results = model.evaluate(validation_set)
print(results)

30/30 [==============================] - 52s 2s/step - loss: 0.6748 - accuracy: 0.8217
[0.6747666597366333, 0.8216560482978821]


In [13]:
predictions = model.predict(test_set)  # Get predictions
predictions = np.argmax(predictions, axis=1)  # Get predicted classes

ground_truth = test_set.classes  # Get ground truth labels

# Calculate confusion matrix
from sklearn.metrics import classification_report
cm = classification_report(ground_truth, predictions)
print(cm)

25/25 [==============================] - 42s 2s/step
              precision    recall  f1-score   support

           0       0.06      0.07      0.06        59
           1       0.11      0.15      0.13        97
           2       0.16      0.16      0.16        95
           3       0.18      0.17      0.18        86
           4       0.05      0.04      0.05        45
           5       0.08      0.04      0.05       102
           6       0.15      0.24      0.19        89
           7       0.05      0.05      0.05        74
           8       0.09      0.08      0.08        66
           9       0.15      0.07      0.10        68

    accuracy                           0.12       781
   macro avg       0.11      0.11      0.10       781
weighted avg       0.11      0.12      0.11       781



In [50]:
predictions = model.predict(validation_set)  # Get predictions
predictions = np.argmax(predictions, axis=1)  # Get predicted classes

ground_truth = validation_set.classes  # Get ground truth labels

# Calculate confusion matrix
from sklearn.metrics import classification_report
cm = classification_report(ground_truth, predictions)
print(cm)

30/30 [==============================] - 59s 2s/step
              precision    recall  f1-score   support

           0       0.78      0.88      0.83        74
           1       0.73      0.78      0.76       106
           2       0.92      0.94      0.93       179
           3       0.95      0.89      0.92        82
           4       0.92      0.97      0.95        72
           5       0.81      0.86      0.83        84
           6       0.75      0.72      0.74        89
           7       0.68      0.81      0.74        74
           8       0.92      0.93      0.92        72
           9       0.85      0.57      0.68       110

    accuracy                           0.83       942
   macro avg       0.83      0.84      0.83       942
weighted avg       0.84      0.83      0.83       942



In [38]:
from tensorflow.keras.preprocessing import image# type: ignore

In [53]:
img=image.load_img(r'C:\Users\Asus\Documents\Ship Classification\flask\input\Disney-Treasure-Exterior-2-1-1.jpg',target_size=(224,224))
img=image.img_to_array(img)
img=img.reshape((1,img.shape[0],img.shape[1],img.shape[2]))
img=preprocess_input(img)
pred=model.predict(img)
pred=pred.flatten()
pred=list(pred)
n=max(pred)
val_dict={0: 'Aircraft Carrier',
 1: 'Bulkers',
 2: 'Car Carrier',
 3: 'Container Ship',
 4: 'Cruise',
 5: 'DDG',
 6: 'Recreational',
 7: 'Sailboat',
 8: 'Submarine',
 9: 'Tug'}
result=val_dict[pred.index(n)]
print(result)

1/1 [==============================] - 0s 104ms/step
Cruise
